# DEPRECATED! GO HERE FOR LATEST WORKING: <a href="../../job/space_track/filter_raw_data_gabbard.ipynb">job/space_track/filter_raw_data_gabbard.ipynb</a>

#### Please leave this alone though!

In [1]:
raise Exception("STOP! THIS IS DEPRECATED!")

Exception: STOP! THIS IS DEPRECATED!

In [ ]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

In [ ]:
csv_store_path = "../../../siads591 data/space_track_raw/csv/"
filtered_store_path = "../../../siads591 data/filtered_raw/"
output_file_name = "gabbard_min.pkl"

In [ ]:
# setting up default and initial variables
columns = ['NORAD_CAT_ID', 'MEAN_MOTION', 'ECCENTRICITY', 'PERIOD', 'APOAPSIS', 'PERIAPSIS', 'OBJECT_TYPE', 'EPOCH']
dtypes = {
    'NORAD_CAT_ID':np.uint16,
    'PERIOD':np.float64,
    'APOAPSIS':np.float64,
    'PERIAPSIS':np.float64,
    'MEAN_MOTION':np.float64,
    'ECCENTRICITY':np.float64,
}

In [ ]:
dfs = []
files = sorted([x for x in os.listdir(f'{csv_store_path}/') if x.endswith(".csv.gz")])
for f in tqdm(files):
    if f.endswith(".csv.gz"):
#        LEO = Mean Motion > 11.25 and Eccentricity < 0.25
        df = pd.read_csv(f'{csv_store_path}{f}', usecols=columns, parse_dates=['EPOCH'], infer_datetime_format=True, index_col='EPOCH', dtype=dtypes, compression='gzip')
        df = df[(df.MEAN_MOTION > 11.25) & (df.ECCENTRICITY < 0.25) & ((df.OBJECT_TYPE == "DEBRIS") | (df.OBJECT_TYPE == "ROCKET BODY"))][['NORAD_CAT_ID', 'PERIOD', 'APOAPSIS', 'PERIAPSIS']]
        
        # For the purpose of generating gabbard diagrams, an APOAPSIS of > 3000 is out of range thus, we can represent
        # APOAPSIS scaled by 20x to a uint16 with a max precision of 1/20 step
        df['APOAPSIS'] = (df['APOAPSIS'] * 20).clip(np.iinfo(np.uint16).min,np.iinfo(np.uint16).max).astype(int).astype(pd.UInt16Dtype())
        
        # similarly PERIAPSIS can be scaled by 20x
        df['PERIAPSIS'] = (df['PERIAPSIS'] * 20).clip(np.iinfo(np.uint16).min,np.iinfo(np.uint16).max).astype(int).astype(pd.UInt16Dtype())
        
        # PERIOD does not exceed 130, and can be scaled by 500x
        df['PERIOD'] = (df['PERIOD'] * 500).clip(np.iinfo(np.uint16).min,np.iinfo(np.uint16).max).astype(int).astype(pd.UInt16Dtype())
        
        # A slight loss of precision for gabbard diagrams are OK because the pixels are close / same in these cases, but
        # being able to use 16bit variants allow much more efficient use of memory and is crucial for being able to perform
        # the interpolation processing step.
        
        # Since animated gabbard diagrams are generated per frame, we can revert the scaling when we plot the graphs
        dfs.append(df)
df = pd.concat(dfs)
df = df.rename(columns = {'PERIOD':'PERIOD_x500', 'APOAPSIS':'APOAPSIS_x20', 'PERIAPSIS':'PERIAPSIS_x20'})
print(f"Total number of rows {len(df)}")
print(f"Exporting to: {filtered_store_path}{output_file_name}.gz")
df.to_pickle(f"{filtered_store_path}{output_file_name}.gz",compression="gzip")
print(f"Export Complete!")